In [1]:
import motrainer
import xarray as xr

## Load data

In [2]:
ds = xr.open_zarr("../example/example1_data.zarr/")
ds

<xarray.Dataset>
Dimensions:    (space: 5, time: 8506)
Coordinates:
    latitude   (space) float64 dask.array<chunksize=(5,), meta=np.ndarray>
    longitude  (space) float64 dask.array<chunksize=(5,), meta=np.ndarray>
  * time       (time) datetime64[ns] 2007-01-02 ... 2020-01-01T01:00:00
Dimensions without coordinates: space
Data variables:
    BIOMA1     (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    BIOMA2     (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    TG1        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    TG2        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    TG3        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    WG1        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    WG2        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    WG3        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    curv       (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    sig        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    slop       (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
Attributes:
    license:  data license
    source:   data source

## Split per gridcell

In [3]:
motrainer.is_splitable(ds)

True

In [4]:
bags = motrainer.dataset_split(ds, "space")
bags

dask.bag<from_sequence, npartitions=5>

## Covert splitted DS to DDF

In [5]:
def to_dataframe(ds):
    return ds.to_dask_dataframe()


def chunk(ds, chunks):
    return ds.chunk(chunks)


test_size = 0.33
f_shuffle = True

bags = bags.map(chunk, {"space": 100}).map(to_dataframe)

## Train Test Split

In [6]:
import dask_ml.model_selection as dcv

train_test_bags = bags.map(
    dcv.train_test_split, test_size=test_size, shuffle=f_shuffle, random_state=1
)  # train_test_split only takes da, df or series. No ds.

In [7]:
train_bags = train_test_bags.pluck(0)
test_bags = train_test_bags.pluck(1)

## Setup Training

In [8]:
# Setup grid search
# use the estimator definition and pipeline objects in sklearn
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RepeatedKFold

from dask_ml.preprocessing import MinMaxScaler
from dask_ml.model_selection import GridSearchCV

regSVR = make_pipeline(MinMaxScaler(), SVR())
kernel = ["poly", "rbf", "sigmoid"]
C = [1, 0.1]
gamma = ["scale"]
grid = dict(svr__kernel=kernel, svr__C=C, svr__gamma=gamma)
cv = RepeatedKFold(n_splits=4, n_repeats=2, random_state=1)
grid_search = GridSearchCV(
    estimator=regSVR,
    param_grid=grid,
    cv=cv,
    scoring=["r2", "neg_mean_squared_error"],
    refit="r2",
)

In [9]:
df_example = train_bags.take(1)[0].compute()

## Model Optimization

In [10]:

def optimize(df, grid_search, input_list, output_list):
    """Customized Optimization Function
    """
    df = df.dropna()
    grid_result = grid_search.fit(df[input_list], df[output_list])
    return grid_result


input_list = ["BIOMA1", "BIOMA1", "TG1", "TG2", "TG3"]
output_list = ["slop"]
optimazed_estimators = train_bags.map(
    optimize, grid_search=grid_search, input_list=input_list, output_list=output_list
)

In [11]:
optimazed_estimators_realized = optimazed_estimators.compute()

/home/oku/miniforge3/envs/vegetation/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/oku/miniforge3/envs/vegetation/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/oku/miniforge3/envs/vegetation/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/oku/miniforge3/envs/vegetation/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-v

## Save model

In [12]:
# To be replaced by "modelstore"
import pickle

for model, id in zip(optimazed_estimators_realized, range(len(optimazed_estimators_realized))):
    name_model = f"model{id}.pickle"
    with open(name_model, "wb") as f:
        pickle.dump(model, f)

## Model performance evaluation

In [13]:
# Load the models back
list_model = []
for id in range(5): 
    with open(f"model{id}.pickle", "rb") as f:
        list_model.append(pickle.load(f))
list_model

[GridSearchCV(cv=RepeatedKFold(n_repeats=2, n_splits=4, random_state=1),
              estimator=Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                                        ('svr', SVR())]),
              param_grid={'svr__C': [1, 0.1], 'svr__gamma': ['scale'],
                          'svr__kernel': ['poly', 'rbf', 'sigmoid']},
              refit='r2', scoring=['r2', 'neg_mean_squared_error']),
 GridSearchCV(cv=RepeatedKFold(n_repeats=2, n_splits=4, random_state=1),
              estimator=Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                                        ('svr', SVR())]),
              param_grid={'svr__C': [1, 0.1], 'svr__gamma': ['scale'],
                          'svr__kernel': ['poly', 'rbf', 'sigmoid']},
              refit='r2', scoring=['r2', 'neg_mean_squared_error']),
 GridSearchCV(cv=RepeatedKFold(n_repeats=2, n_splits=4, random_state=1),
              estimator=Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                       

In [14]:
from sklearn.metrics import mean_squared_error , r2_score,  mean_absolute_error

# This for need to be coverted to a user defined 
list_metrics = []
for model, test_data in zip(list_model, test_bags.compute()):
    test_data = test_data.dropna()
    X_test = test_data[input_list]
    Y_test = test_data[output_list]
    Y_eval = model.predict(X_test)

    metrics = {"MSE_SVR": mean_squared_error(Y_test,Y_eval),
               "MAE_SVR": mean_absolute_error(Y_test,Y_eval),
               "R_2":r2_score(Y_test,Y_eval)}
    list_metrics.append(metrics)